In [ ]:

import os
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import azure.cognitiveservices.speech as speechsdk

speech_key = "C2aQwIVI4DwKew11iZqZiOn4x1FEt7qgaM2qIfDZIdCXnZm9LEfMJQQJ99BEACYeBjFXJ3w3AAAEACOGfGDZ"
service_region = "eastus"

speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
speech_config.speech_synthesis_voice_name = "sw-KE-ZuriNeural"
synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)

watch_folder = "sound"

class NewTextFileHandler(FileSystemEventHandler):
    def on_created(self, event):
        # Only handle files, ignore directories
        if not event.is_directory and event.src_path.endswith('.txt'):
            print(f"New file detected: {event.src_path}")
            self.read_and_speak(event.src_path)
    
    def read_and_speak(self, filepath):
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                text = f.read().strip()
            if text:
                print(f"Speaking text: {text}")
                result = synthesizer.speak_text_async(text).get()
                if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
                    print("Speech synthesis completed.")
                else:
                    print(f"Speech synthesis error: {result.reason}")
        except Exception as e:
            print(f"Error reading/speaking file: {e}")

if __name__ == "__main__":
    if not os.path.exists(watch_folder):
        os.makedirs(watch_folder)

    event_handler = NewTextFileHandler()
    observer = Observer()
    observer.schedule(event_handler, path=watch_folder, recursive=False)
    observer.start()
    print(f"Watching folder: {watch_folder}")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()


Watching folder: sound
New file detected: sound\output_20250614_034426.txt
Speaking text: BABA
Speech synthesis completed.


In [2]:
%pip install watchdog

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install watchdog azure-cognitiveservices-speech

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ----------------- ---------------------- 1.0/2.4 MB 3.0 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.4 MB 3.0 MB/s eta 0:00:01
   -------------------------------------- - 2.4/2.4 MB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 2.9 MB/s eta 0:00:00

   ---------------------------------------- 0/7 [urllib3]
   ---------------------------------------- 0/7 [urllib3]
   ---------------------------------------- 0/7 [urllib3]
   ---------------------------------------- 0/7 [urllib3]
   ----- ---------------------------------- 1/7 [idna]
   ----- ---------------------------------- 1/7 [idna]
   ----------- ---------------------------- 2/7 [charset_normalizer]
   ----------- ---------------------------- 2/7 [charset_normalizer]
   -----